In [ ]:
import os


!pip install keras4torch
!pip install einops



import keras4torch
from   keras4torch.callbacks  import ModelCheckpoint,LRScheduler
import torch
import torch.nn    as nn
import torch.optim as optim
import  torch.nn.functional as     F
import numpy       as np
import pandas      as pd
from copy import deepcopy
import  matplotlib.pyplot   as     plt
from    sklearn.preprocessing import StandardScaler, QuantileTransformer
from    datetime import datetime
import  gc
import STab
from STab import MyClassLoss,CatMap,Num_Cat
from   STab import mainmodel, LWTA, Gsoftmax
MainModel=mainmodel.MainModel

In [ ]:
#Split the data 
X_test=pd.DataFrame(np.load('Data/jannis/N_test.npy')).astype(np.float64).fillna(0)
y_test=pd.DataFrame(np.load('Data/jannis/y_test.npy')).astype(np.int32)

X_train=pd.DataFrame(np.load('Data/jannis/N_train.npy')).astype(np.float64).fillna(0)
Y_train=pd.DataFrame(np.load('Data/jannis/y_train.npy')).astype(np.int32)


X_valid=pd.DataFrame(np.load('Data/jannis/N_val.npy')).astype(np.float64).fillna(0)
y_valid=pd.DataFrame(np.load('Data/jannis/y_val.npy')).astype(np.int32)

In [ ]:
#Normalise In
scale_factor=X_train.max()

X_test  = (X_test/scale_factor).values
X_valid = (X_valid/scale_factor).values
X_train = (X_train/scale_factor).values

In [ ]:
checkpoint='saved/savefileJA'

In [ ]:
def True_ACC(p,t):
                return np.mean(np.where(np.round(p)==np.round(t),1,0))

In [ ]:
Or_model = MainModel(
    categories        = (),
    num_continuous    = 54,
    dim               = 192  ,
    dim_out           = 4,
    depth             = 4,
    heads             = 8,
    attn_dropout      = 0.25 ,
    ff_dropout        = 0.25,
    U                 = 2,
    cases             = 16

)
no_model = Num_Cat(Or_model,num_number=54,classes=4)
model    = keras4torch.Model(no_model,).build([54])


#Warm UP
optimizer=torch.optim.AdamW(model.parameters(),lr=0.0001,weight_decay=0.0001,)
sch=torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.001, total_iters=10,  verbose=False)
model.compile(optimizer=optimizer, loss=MyClassLoss(0.01,1), metrics=['accuracy', F.cross_entropy])
callbacks=[LRScheduler(sch)]
model.fit(X_train, Y_train[0].values,
                      epochs=10, batch_size=512,
                      validation_data=(X_valid,y_valid[0].values),
                      verbose=2,validation_batch_size=128,
                      callbacks=callbacks)

#Main Train
optimizer=torch.optim.AdamW(model.parameters(),lr=0.001,weight_decay=0.0001,)
model.compile(optimizer=optimizer, loss=MyClassLoss(0.01,1), metrics=['accuracy', F.cross_entropy])
scheduler =LRScheduler( torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5,min_lr=0.00001))
callbacks=[ModelCheckpoint(checkpoint,monitor='val_acc',mode='max'),scheduler]
model.fit(X_train, Y_train[0].values,
        epochs=180, batch_size=512,
        validation_data=(X_valid,y_valid[0].values),
        verbose=2,validation_batch_size=128,
        callbacks=callbacks)



model.load_weights(checkpoint)

logits=0

for i in range(0,64):
        logits+=pd.DataFrame(model.predict(X_test,batch_size=128))

Test = (True_ACC(logits.idxmax(axis=1).values.reshape((-1,1)),y_test))

print('Test: ',Test)